In [1]:
def createC1(D, minSup):#generate prefix array
    C1 = []
    postfixDic={}  
    lenD = len(D)

    for i in range(lenD):
        for idx, item in enumerate(D[i]):
            if tuple([item]) not in C1:
                postfixDic[tuple([item])]={}
                
                C1.append(tuple([item]))
            if i not in postfixDic[tuple([item])].keys():
                postfixDic[tuple([item])][i]=idx

    L1, postfixDic = rmLowSup(D, C1, postfixDic, minSup)              
    
    return L1, postfixDic

In [2]:
def rmLowSup(D,Cx, postfixDic,minSup):#remove those with low support
    Lx = Cx
    for iset in Cx:
        if len(postfixDic[iset])/len(D) < minSup:
            Lx.remove(iset)
            postfixDic.pop(iset)
        
    return Lx, postfixDic

In [3]:
def psGen(D, Lk, postfixDic, minSup, minConf): # generate new prefix array

    retList = []
    lenD = len(D) 
    for Ck in Lk:
        item_count = {} 
        for i in postfixDic[Ck].keys(): 
            item_exsit={}
            for j in range(postfixDic[Ck][i]+1, len(D[i])):
                if D[i][j] not in item_count.keys():
                    item_count[D[i][j]]=0
                if D[i][j] not in item_exsit:
                    item_count[D[i][j]]+=1
                    item_exsit[D[i][j]]=True

        c_items = []
 
        for item in item_count.keys():
            if item_count[item]/lenD >= minSup and item_count[item]/len(postfixDic[Ck])>=minConf:
                c_items.append(item)
         
        for c_item in c_items:
            retList.append(Ck+tuple([c_item]))
    
    return retList

In [4]:
def genNewPostfixDic(D,Lk, prePost):# generate new postfix array
    
    postfixDic = {}

    for Ck in Lk:
        # (1,2)的后缀只能出现在 (1,)的后缀列表里
        postfixDic[Ck]={}
        tgt = Ck[-1]
        prePostList = prePost[Ck[:-1]]
        for r_i in prePostList.keys():
            for c_i in range(prePostList[r_i]+1, len(D[r_i])):
                if D[r_i][c_i]==tgt:
                    postfixDic[Ck][r_i] = c_i
                    break
    
    return postfixDic

In [5]:
def PrefixSpanAlogrithm(D, minSup=0.5, minConf=0.5):
    L1, postfixDic = createC1(D, minSup)
    L = []
    k = 2

    Dic_L1 = [{x:len(postfixDic[x])} for x in L1]
    Dic_L = [Dic_L1]
    L = [L1]
    
    while len(L[k-2]) > 0:
        
        Lk = psGen(D, L[k-2], postfixDic, minSup, minConf)
        
        postfixDic = genNewPostfixDic(D,Lk, postfixDic)
        Dic_Lk = [{x:len(postfixDic[x])} for x in Lk]

        Dic_L1.append(Dic_Lk)
        
        L.append(Lk)
        k+=1
    return Dic_L

In [6]:
D = [
    [1,4,2,3],
    [0, 1, 2, 3],
    [1, 2, 1, 3, 4],
    [2, 1, 2, 4, 4],
    [1, 1, 1, 2, 3],
]
L = PrefixSpanAlogrithm(D,minSup=0.3,minConf=0.3)
for i in range(len(L)-1,-1,-1):
    for Cx in L[i]:
        print(Cx)

{(1,): 5}
{(4,): 3}
{(2,): 5}
{(3,): 4}
[{(1, 4): 3}, {(1, 2): 5}, {(1, 3): 4}, {(1, 1): 2}, {(2, 3): 4}, {(2, 1): 2}, {(2, 4): 2}]
[{(1, 2, 3): 4}, {(1, 2, 4): 2}, {(1, 1, 3): 2}, {(2, 1, 4): 2}]
[]


In [7]:
import pandas as pd
import numpy as np

aisle = pd.read_csv('../../DataSets/aisles.csv')
department = pd.read_csv('../../DataSets/departments.csv')
order_products_prior = pd.read_csv('../../DataSets/order_products__prior.csv')
order_products_train = pd.read_csv('../../DataSets/order_products__train.csv')
orders = pd.read_csv('../../DataSets/orders.csv')
products = pd.read_csv('../../DataSets/products.csv')

# fresh fruits:24
# fresh vegetables:83
# packaged vegetables fruits:122


### find the top 3 aisles [fresh fruits,fresh vegetables,packaged vegetables fruits]

In [8]:
aisle_ids = []
aisle_arr = aisle[aisle['aisle'].isin(['fresh fruits','fresh vegetables','packaged vegetables fruits'])].values
for item in aisle_arr:
    aisle_ids.append(item[0])

product_ids = []
product_arr = products[products['aisle_id'].isin(aisle_ids)].values
for item in product_arr:
    product_ids.append(item[0])
product_ids
aisle_ids

[24, 83, 123]

In [9]:
order_products_id1 = order_products_train[['order_id','product_id','add_to_cart_order']]
order_products_id2 = order_products_prior[['order_id','product_id','add_to_cart_order']]
orderId_productsId_order = order_products_id1.append(order_products_id2, ignore_index=True)
# orderId_productsId_order = order_products_id2
orderId_productsId_order = orderId_productsId_order[~orderId_productsId_order['product_id'].isin(product_ids)]

In [10]:
len(orderId_productsId_order)

24614009

In [11]:
orderId_productsId_order = orderId_productsId_order[orderId_productsId_order['add_to_cart_order']<20]
orderId_productsId_order

,order_id,product_id,add_to_cart_order
0,1,49302,1
1,1,11109,2
4,1,43633,5
7,1,22035,8
8,36,39612,1
...,...,...,...
33819100,3421083,35211,5
33819101,3421083,39678,6
33819102,3421083,11352,7
33819103,3421083,4600,8


In [12]:
22598417/24614009

0.9181119987402296

In [13]:
user_classification = pd.read_csv('../customer_classificationV2.csv')
user_classification = user_classification[['user_id','label']]
user1 = user_classification[user_classification['label']==0]
user2 = user_classification[user_classification['label']==1]
user3 = user_classification[user_classification['label']==2]
user4 = user_classification[user_classification['label']==3]
user5 = user_classification[user_classification['label']==4]


In [14]:
def getTable(user1):
    orders_no_test = orders[orders['eval_set']!='test']
    order_table1 = orders_no_test[orders_no_test['user_id'].isin(user1['user_id'])]

    orderId_productsId_order1=orderId_productsId_order[orderId_productsId_order['order_id'].isin(order_table1['order_id'])]

    new_table1 = pd.pivot_table(orderId_productsId_order1,values='product_id',index=['order_id'],columns=['add_to_cart_order'],fill_value=0)
    
    arr1=[]
    for item in new_table1.values:
        arr1.append(item[item!=0])
    return arr1

In [15]:
# order_products_id1 = order_products_train[['order_id','product_id','add_to_cart_order']]
# order_products_id2 = order_products_prior[['order_id','product_id','add_to_cart_order']]
# orderId_productsId_order_all = order_products_id1.append(order_products_id2, ignore_index=True)
# new_table_all = pd.pivot_table(orderId_productsId_order_all,values='product_id',index=['order_id'],columns=['add_to_cart_order'],fill_value=0)
# arr_all=[]
# for item in new_table_all.values:
#     arr_all.append(item[item!=0])

In [16]:
arr_all=getTable(user_classification)

In [17]:
arr1=getTable(user1)
arr1

[array([33754, 24838, 17668, 17461, 32665]),
 array([23423]),
 array([31717,  1529, 22122, 34134, 27156, 49235, 26842]),
 array([20392, 27845,   162,  2452,  8575, 41890, 39475, 10096, 23032,
        20995]),
 array([ 5009, 33106, 31478, 25718]),
 array([23909, 22754, 23341, 46941, 19068, 21410, 17341, 40593, 31268]),
 array([40078,  9065, 46175]),
 array([35951, 25890, 33120, 40545]),
 array([35108, 40593, 17461, 25256, 37220, 27548, 45504, 18531, 24024,
        38544]),
 array([35561, 33059, 43721]),
 array([27344, 27355]),
 array([19887, 33000]),
 array([20307, 19677]),
 array([  397, 34993, 17183, 37788,  1025,  4799,   276]),
 array([18963,  4485]),
 array([36606, 32442, 14467, 22298, 38827, 38647, 34448, 46194]),
 array([ 7021,  4799, 11520,  9076,  1376, 21910, 34050]),
 array([47231, 48857]),
 array([23375,  2825]),
 array([ 9934, 22395, 36929, 37110,  2298, 48628,   781, 13249,   890,
        22963]),
 array([25886, 41674,  9327, 24203, 34854, 34640,  6046,   287]),
 array([17

In [18]:
arr2=getTable(user2)
arr2

[array([ 8021,  1000, 36011, 44632, 38985,  7948,  4802, 28934,  2116,
        45495, 32018, 27644,  4656]),
 array([43875, 27845, 47049,  8186, 34134, 42345, 28682, 36342, 12341,
        23060]),
 array([16185, 33731, 35715, 47888, 29077, 31640, 19240, 14211, 26301,
        11520, 30252, 30489]),
 array([ 3025,  5479,  1516, 44349, 28934, 32433, 46346, 48720, 28849]),
 array([45368, 28682, 29987, 39812, 31343, 38885]),
 array([35561, 46955, 39560, 12144, 45633,  5785,   162, 45767, 35851,
        48726,  4663, 47144]),
 array([27086, 16773, 45437, 47144, 39409, 20345, 43802, 43192, 46049,
        47668,  4656, 48171, 37158, 10166, 26346, 42736]),
 array([31504, 14643, 13198, 27845, 30994,  3874, 25824, 37119, 33593,
        45681,  1215, 37029]),
 array([ 6278, 42538, 29615, 44042, 12782,  7241, 24504,  6052, 25830,
        42736, 25804, 47185, 12807, 26767, 39910]),
 array([ 4781, 13780,  5769,  9036, 30290, 16145, 20316,  7485, 17200,
        12409, 20957, 39971]),
 array([33240,   

In [19]:
arr3=getTable(user3)
arr3

[array([46842, 26434, 39758, 27761, 10054, 21351, 22598, 34862, 40285,
        17616, 25146, 32645, 41276]),
 array([30597, 15221, 37886, 37215, 34335,  3164, 26910, 38888, 38050,
        25213, 11175, 23390, 29471, 43511]),
 array([11422, 27514, 46654, 47993,  5438, 46226, 25670]),
 array([49510, 36481,  5120, 43154,  2979, 23233, 48057, 31040, 38944,
        39619, 38273]),
 array([ 1463, 11193, 34423, 28842, 13984, 42300, 17594, 34065, 18306]),
 array([19511, 35628, 18523,  2067,  7485,  9366, 30442, 49131, 46654,
        27690, 27548,  1158, 26252]),
 array([44116,  4793, 39121, 29742, 18288, 45633,  2929,  1376, 35503,
        14332, 35704, 48385, 35561,  1392]),
 array([ 8859, 19731, 43654,  4357, 37664, 34065, 35951, 43560,  9896,
        27509, 15455, 47601, 40396, 35042, 40986,  1939, 46313]),
 array([31577, 47783, 43394, 16953, 11513]),
 array([36217,  8195, 25146,  7948,  4715, 10616, 18848, 26165, 14131,
        19076, 35871, 49056, 31555, 22894, 37751, 42240, 15452, 33651]

In [20]:
arr4=getTable(user4)
arr4

[array([43352, 21386,  8571]),
 array([ 8571, 21292]),
 array([12078, 43352, 26348, 41400, 30591, 19660, 37710, 13914]),
 array([ 9076, 43352]),
 array([43154, 12341]),
 array([42224,  6184]),
 array([25802, 21292, 12341, 43352]),
 array([20940, 12341, 20955, 25133,  8048, 32455, 21292, 43352]),
 array([11365, 43352]),
 array([20448]),
 array([8048]),
 array([ 8048, 12341, 21573, 11759, 30450, 39332,  9076]),
 array([20940, 12341, 37919, 27984, 19691, 21385, 33198, 30827,  8803,
        19660,  5782]),
 array([ 8571, 46149,   196, 14715, 13424, 21572, 11266, 26104, 13042,
        30486,  7987]),
 array([27845, 19286, 34551,   196]),
 array([39657, 16974, 12341,  6184, 37710, 10441, 43352, 49235, 32455,
        25146]),
 array([18615, 19508]),
 array([43154,  6760, 43352, 19691, 38928]),
 array([10623,  8048]),
 array([32478, 25890]),
 array([ 4724, 13535, 46473]),
 array([30827, 46129, 33081,  7862]),
 array([14032, 44449]),
 array([30827,  8048, 20940, 15541, 19691]),
 array([43352,  

In [21]:
arr5=getTable(user5)
arr5

[array([15005, 47329, 23909, 48370, 13245,  9633, 27360,  6348, 40878,
         6184, 48002, 20914, 12962, 45698, 24773, 18569]),
 array([19660, 21195,  7461,  2996, 32463]),
 array([10070, 42450, 33198, 34866]),
 array([45504, 35003, 23955, 22920, 23375, 39636, 48590, 37358,  1186,
        40332, 49199]),
 array([18908, 22395, 23400, 17206, 29987, 31506, 28425, 12572]),
 array([21131, 16696, 43616, 42972,  6991,  1694, 36316, 23909, 34185,
        43693, 32650, 30913, 13641, 13627, 10957]),
 array([14127, 45645, 21506]),
 array([44422,  7503, 30884, 11819, 32016,  1463, 14332, 41257, 19184]),
 array([ 5479, 29650,  7948,  5161, 41757, 26324, 39261, 10286,    86]),
 array([46064, 27156,  9390,  9076, 46802,  1158, 28553, 26165, 11941]),
 array([32850, 49520, 26521, 10831,  8667,  3849, 14947]),
 array([3919]),
 array([44632, 24838, 44257, 45645, 31824, 25890, 41774, 10132]),
 array([ 3957,  1169, 43295,  6568,  6287, 20947, 17333, 20794, 44632,
        44422, 11688, 22227, 33462]),
 ar

# implemented by algorithm(not used)

In [22]:

# L = PrefixSpanAlogrithm(getTable(user1)[:30000],minSup=0.3,minConf=0.3)
# for i in range(len(L)-1,-1,-1):
#     for Cx in L[i]:
#         print(Cx)

In [23]:
# L = PrefixSpanAlogrithm(arr2[:30000],minSup=0.3,minConf=0.3)
# for i in range(len(L)-1,-1,-1):
#     for Cx in L[i]:
#         print(Cx)

In [24]:
# L = PrefixSpanAlogrithm(arr3[:30000],minSup=0.3,minConf=0.3)
# for i in range(len(L)-1,-1,-1):
#     for Cx in L[i]:
#         print(Cx)

In [25]:
# L = PrefixSpanAlogrithm(arr4[:30000],minSup=0.3,minConf=0.3)
# for i in range(len(L)-1,-1,-1):
#     for Cx in L[i]:
#         print(Cx)

# implemented by third-party package

In [26]:
def transform_to_product(association_rule):
    order_rules = []
    for item in association_rule:
        rule = []
        for product in item:
            rule.append(products[products['product_id' ]== product]['product_name'].values[0])
        order_rules.append(rule)
    return order_rules

In [27]:
from prefixspan import PrefixSpan

# all user

In [28]:
ps_all = PrefixSpan(arr_all)
top_table_all=ps_all.topk(1000)

In [29]:
top_rule_all=[]
for item in top_table_all:
    if(len(item[1])>1):
        top_rule_all.append(item[1])
len(top_rule_all)
transform_to_product(top_rule_all)

[['Sparkling Water Grapefruit', 'Lime Sparkling Water'],
 ['Sparkling Water Grapefruit', 'Sparkling Lemon Water'],
 ['Total 2% with Strawberry Lowfat Greek Strained Yogurt',
  'Total 2% Lowfat Greek Strained Yogurt With Blueberry'],
 ['Organic Whole Milk', 'Organic Whole String Cheese'],
 ['Lime Sparkling Water', 'Sparkling Lemon Water'],
 ['Lime Sparkling Water', 'Sparkling Water Grapefruit'],
 ['Total 2% with Strawberry Lowfat Greek Strained Yogurt',
  'Total 2% Lowfat Greek Strained Yogurt with Peach'],
 ['Sparkling Water Grapefruit', 'Peach Pear Flavored Sparkling Water'],
 ['Sparkling Water Grapefruit', 'Sparkling Water Berry'],
 ['Organic Whole Milk', 'Organic Half & Half'],
 ['Total 2% Lowfat Greek Strained Yogurt With Blueberry',
  'Total 2% with Strawberry Lowfat Greek Strained Yogurt'],
 ['Icelandic Style Skyr Blueberry Non-fat Yogurt', 'Non Fat Raspberry Yogurt'],
 ['Sparkling Water Grapefruit', 'Pure Sparkling Water']]

# user classification 0

In [30]:
ps1 = PrefixSpan(arr1)

top_table1=ps1.topk(1000)
# top_table1

In [31]:
top_rule1=[]
for item in top_table1:
    if(len(item[1])>1):
        top_rule1.append(item[1])
len(top_rule1)

9

In [32]:
transform_to_product(top_rule1)

[['Organic Whole Milk', 'Organic Whole String Cheese'],
 ['Total 2% with Strawberry Lowfat Greek Strained Yogurt',
  'Total 2% Lowfat Greek Strained Yogurt With Blueberry'],
 ['Total 2% with Strawberry Lowfat Greek Strained Yogurt',
  'Total 2% Lowfat Greek Strained Yogurt with Peach'],
 ['Organic Whole Milk', 'Organic Half & Half'],
 ['Organic Cilantro', 'Organic Italian Parsley Bunch'],
 ['Organic Whole Milk', 'Organic Cilantro'],
 ['Organic Whole Milk', 'Original Hummus'],
 ['Organic Whole Milk', 'Whole Milk Plain Yogurt'],
 ['Icelandic Style Skyr Blueberry Non-fat Yogurt', 'Non Fat Raspberry Yogurt']]

# user classification 1

In [33]:
ps2 = PrefixSpan(arr2)
top_table2 = ps2.topk(1000)
# top_table2

In [34]:
top_rule2=[]
for item in top_table2:
    if(len(item[1])>1):
        top_rule2.append(item[1])
len(top_rule2)

131

In [35]:
transform_to_product(top_rule2)

[['Organic Pears, Peas and Broccoli Puree Stage 1',
  'Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food'],
 ['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food',
  'Organic Apples, Carrots and Parsnips Puree'],
 ['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food',
  'Organic 4 Months Butternut Squash Carrots Apples + Prunes Baby Food'],
 ['Spinach Peas & Pear Stage 2 Baby Food',
  'Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
 ['Spinach Peas & Pear Stage 2 Baby Food',
  'Broccoli & Apple Stage 2 Baby Food'],
 ['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food',
  'Organic Pears, Peas and Broccoli Puree Stage 1'],
 ['Baby Food Stage 2 Blueberry Pear & Purple Carrot',
  'Spinach Peas & Pear Stage 2 Baby Food'],
 ['Organic Pears, Peas and Broccoli Puree Stage 1',
  'Organic 4 Months Butternut Squash Carrots Apples + Prunes Baby Food'],
 ['Baby Food Stage 2 Blueberry Pear &

# user classification 2

In [36]:
ps3 = PrefixSpan(arr3)
top_table3 = ps3.topk(1000)
# top_table3

In [37]:
top_rule3=[]
for item in top_table3:
    if(len(item[1])>1):
        top_rule3.append(item[1])
len(top_rule3)

10

In [38]:
transform_to_product(top_rule3)

[['Total 2% with Strawberry Lowfat Greek Strained Yogurt',
  'Total 2% Lowfat Greek Strained Yogurt With Blueberry'],
 ['Total 2% with Strawberry Lowfat Greek Strained Yogurt',
  'Total 2% Lowfat Greek Strained Yogurt with Peach'],
 ['Organic Whole Milk', 'Organic Whole String Cheese'],
 ['Trilogy Kombucha Drink', 'Synergy Organic Kombucha Gingerberry'],
 ['Organic Raw Kombucha Gingerade', 'Trilogy Kombucha Drink'],
 ['Trilogy Kombucha Drink', 'Organic Raw Kombucha Gingerade'],
 ['Total 2% Lowfat Greek Strained Yogurt With Blueberry',
  'Total 2% with Strawberry Lowfat Greek Strained Yogurt'],
 ['Total 2% with Strawberry Lowfat Greek Strained Yogurt',
  'Total 2% All Natural Greek Strained Yogurt with Honey'],
 ['Total 2% with Strawberry Lowfat Greek Strained Yogurt',
  'Total 2% Greek Strained Yogurt with Cherry 5.3 oz'],
 ['Organic Reduced Fat 2% Milk', 'Half & Half']]

# user classification 3

In [39]:
ps4 = PrefixSpan(arr4)
top_table4 = ps4.topk(500)
# top_table4

In [40]:
top_rule4=[]
for item in top_table4:
    if(len(item[1])>1):
        top_rule4.append(item[1])
len(top_rule4)

254

In [41]:
transform_to_product(top_rule4)

[['Hass Avocados', 'Clementines'],
 ['Hass Avocados', 'Raspberries'],
 ['Raspberries', 'Hass Avocados'],
 ['Raspberries', 'Clementines'],
 ['Clementines', 'Hass Avocados'],
 ['Hass Avocados', 'Packaged Grape Tomatoes'],
 ['Baby Cucumbers', 'Packaged Grape Tomatoes'],
 ['Baby Cucumbers', 'Hass Avocados'],
 ['Hass Avocados', 'Baby Cucumbers'],
 ['Clementines', 'Raspberries'],
 ['Raspberries', 'Blueberries'],
 ['Packaged Grape Tomatoes', 'Hass Avocados'],
 ['Baby Cucumbers', 'Raspberries'],
 ['Hass Avocados', 'Seedless Cucumbers'],
 ['Raspberries', 'Packaged Grape Tomatoes'],
 ['Packaged Grape Tomatoes', 'Baby Cucumbers'],
 ['Hass Avocados', 'Soda'],
 ['Raspberries', 'Seedless Cucumbers'],
 ['Packaged Grape Tomatoes', 'Raspberries'],
 ['Packaged Grape Tomatoes', 'Seedless Cucumbers'],
 ['Packaged Grape Tomatoes', 'Rainbow Bell Peppers'],
 ['Hass Avocados', 'Rainbow Bell Peppers'],
 ['Packaged Grape Tomatoes', 'Clementines'],
 ['Raspberries', 'Baby Cucumbers'],
 ['Seedless Cucumbers', 'Pac

# user classification 4

In [42]:
ps5 = PrefixSpan(arr5)
top_table5 = ps5.topk(1000)
# top_table5

In [43]:
top_rule5=[]
for item in top_table5:
    if(len(item[1])>1):
        top_rule5.append(item[1])
len(top_rule5)

150

In [44]:
transform_to_product(top_rule5)

[['Sparkling Water Grapefruit', 'Lime Sparkling Water'],
 ['Sparkling Water Grapefruit', 'Sparkling Lemon Water'],
 ['Lime Sparkling Water', 'Sparkling Lemon Water'],
 ['Lime Sparkling Water', 'Sparkling Water Grapefruit'],
 ['Sparkling Water Grapefruit', 'Peach Pear Flavored Sparkling Water'],
 ['Sparkling Water Grapefruit', 'Sparkling Water Berry'],
 ['Sparkling Water Grapefruit', 'Pure Sparkling Water'],
 ['Pure Sparkling Water', 'Sparkling Water Grapefruit'],
 ['Sparkling Lemon Water', 'Lime Sparkling Water'],
 ['Sparkling Lemon Water', 'Sparkling Water Grapefruit'],
 ['Lime Sparkling Water', 'Sparkling Water Berry'],
 ['Lime Sparkling Water', 'Peach Pear Flavored Sparkling Water'],
 ['Sparkling Lemon Water', 'Sparkling Water Berry'],
 ['Pure Sparkling Water', 'Sparkling Lemon Water'],
 ['Pure Sparkling Water', 'Lime Sparkling Water'],
 ['Lime Sparkling Water', 'Pure Sparkling Water'],
 ['Sparkling Lemon Water', 'Pure Sparkling Water'],
 ['Sparkling Water Grapefruit',
  'Lime Spark

# find top 20 item

In [45]:
def top20Items(ps):
    top10_item = []
    top10_table = ps.topk(20)
    for item in top10_table:
        tup = (item[1][0],products[products['product_id' ]== item[1][0]]['product_name'].values[0])
        top10_item.append(tup)
    return top10_item

## top 20 item in all user

In [46]:
top20Items(ps_all)

[(27845, 'Organic Whole Milk'),
 (49235, 'Organic Half & Half'),
 (27086, 'Half & Half'),
 (44632, 'Sparkling Water Grapefruit'),
 (30489, 'Original Hummus'),
 (22035, 'Organic Whole String Cheese'),
 (31717, 'Organic Cilantro'),
 (35951, 'Organic Unsweetened Almond Milk'),
 (34126, 'Organic Italian Parsley Bunch'),
 (5077, '100% Whole Wheat Bread'),
 (19660, 'Spring Water'),
 (5785, 'Organic Reduced Fat 2% Milk'),
 (43352, 'Raspberries'),
 (24838, 'Unsweetened Almondmilk'),
 (12341, 'Hass Avocados'),
 (9076, 'Blueberries'),
 (31506, 'Extra Virgin Olive Oil'),
 (35221, 'Lime Sparkling Water'),
 (33731, 'Grated Parmesan'),
 (25890, 'Boneless Skinless Chicken Breasts')]

## top 20 item in user classification 0

In [47]:
top20Items(ps1)

[(27845, 'Organic Whole Milk'),
 (31717, 'Organic Cilantro'),
 (49235, 'Organic Half & Half'),
 (27086, 'Half & Half'),
 (34126, 'Organic Italian Parsley Bunch'),
 (30489, 'Original Hummus'),
 (35951, 'Organic Unsweetened Almond Milk'),
 (22035, 'Organic Whole String Cheese'),
 (5077, '100% Whole Wheat Bread'),
 (5785, 'Organic Reduced Fat 2% Milk'),
 (31506, 'Extra Virgin Olive Oil'),
 (25890, 'Boneless Skinless Chicken Breasts'),
 (38689, 'Organic Reduced Fat Milk'),
 (9076, 'Blueberries'),
 (33731, 'Grated Parmesan'),
 (24838, 'Unsweetened Almondmilk'),
 (28842, 'Bunched Cilantro'),
 (11520, 'Large Alfresco Eggs'),
 (4210, 'Whole Milk'),
 (18465, 'Organic Grade A Free Range Large Brown Eggs')]

## top 20 item in user classification 1

In [48]:
top20Items(ps2)

[(27845, 'Organic Whole Milk'),
 (22035, 'Organic Whole String Cheese'),
 (43875, 'Baby Food Stage 2 Blueberry Pear & Purple Carrot'),
 (5491,
  'Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food'),
 (34134, 'Spinach Peas & Pear Stage 2 Baby Food'),
 (3020, 'Broccoli & Apple Stage 2 Baby Food'),
 (49044, 'Organic Pears, Peas and Broccoli Puree Stage 1'),
 (5077, '100% Whole Wheat Bread'),
 (30489, 'Original Hummus'),
 (7781, 'Organic Sticks Low Moisture Part Skim Mozzarella String Cheese'),
 (44632, 'Sparkling Water Grapefruit'),
 (5785, 'Organic Reduced Fat 2% Milk'),
 (49235, 'Organic Half & Half'),
 (14211, 'Baby Food Stage 2 Pumpkin Banana'),
 (33731, 'Grated Parmesan'),
 (21162, 'Organic Mixed Berry Yogurt & Fruit Snack'),
 (4812, 'Stage 1 Apples & Strawberries Organic Pureed Baby Food'),
 (5114, 'Peach,  Apricot & Banana Stage 2 Baby Food'),
 (47888, 'Apple and Carrot Stage 2 Baby Food'),
 (18465, 'Organic Grade A Free Range Large Brown Eggs')]

## top 20 item in user classification 2

In [49]:
top20Items(ps3)

[(27845, 'Organic Whole Milk'),
 (22035, 'Organic Whole String Cheese'),
 (30489, 'Original Hummus'),
 (27086, 'Half & Half'),
 (49235, 'Organic Half & Half'),
 (3957, '100% Raw Coconut Water'),
 (5077, '100% Whole Wheat Bread'),
 (35951, 'Organic Unsweetened Almond Milk'),
 (24838, 'Unsweetened Almondmilk'),
 (27344, 'Uncured Genoa Salami'),
 (30776, 'Organic Raw Kombucha Gingerade'),
 (5785, 'Organic Reduced Fat 2% Milk'),
 (45603, 'Trilogy Kombucha Drink'),
 (44632, 'Sparkling Water Grapefruit'),
 (7781, 'Organic Sticks Low Moisture Part Skim Mozzarella String Cheese'),
 (22963, 'Organic Roasted Turkey Breast'),
 (11520, 'Large Alfresco Eggs'),
 (31717, 'Organic Cilantro'),
 (9076, 'Blueberries'),
 (33731, 'Grated Parmesan')]

## top 20 item in user classification 3

In [50]:
top20Items(ps4)

[(12341, 'Hass Avocados'),
 (43352, 'Raspberries'),
 (6184, 'Clementines'),
 (8571, 'Baby Cucumbers'),
 (8048, 'Packaged Grape Tomatoes'),
 (30827, 'Seedless Cucumbers'),
 (21292, 'Rainbow Bell Peppers'),
 (196, 'Soda'),
 (38768, 'Sweet Kale Salad Mix'),
 (38928, '0% Greek Strained Yogurt'),
 (19691, 'Green Seedless Grapes'),
 (32478, 'Reduced Fat 2% Milk'),
 (9076, 'Blueberries'),
 (4724, 'Broccoli Florettes'),
 (35561, 'Organic Tortilla Chips'),
 (31805, 'Gourmet Tomato Medley'),
 (20940, 'Organic Low Fat Milk'),
 (31651, 'Extra Fancy Unsalted Mixed Nuts'),
 (37710, 'Trail Mix'),
 (11759, 'Organic Simply Naked Pita Chips')]

## top 20 item in user classification 4

In [51]:
top20Items(ps5)

[(44632, 'Sparkling Water Grapefruit'),
 (19660, 'Spring Water'),
 (35221, 'Lime Sparkling Water'),
 (21709, 'Sparkling Lemon Water'),
 (33198, 'Sparkling Natural Mineral Water'),
 (14947, 'Pure Sparkling Water'),
 (5479, 'Italian Sparkling Mineral Water'),
 (49235, 'Organic Half & Half'),
 (27086, 'Half & Half'),
 (27845, 'Organic Whole Milk'),
 (26620, 'Peach Pear Flavored Sparkling Water'),
 (20119, 'Sparkling Water Berry'),
 (26165, 'Electrolyte Enhanced Water'),
 (30489, 'Original Hummus'),
 (5785, 'Organic Reduced Fat 2% Milk'),
 (8859, 'Natural Spring Water'),
 (49520, 'Orange Sparkling Water'),
 (43154, 'Sparkling Mineral Water'),
 (47977, 'Grapefruit Sparkling Water'),
 (35951, 'Organic Unsweetened Almond Milk')]